<a href="https://colab.research.google.com/github/zingarelli/alura-challenges-data-science-2022/blob/main/Alura_Challenges_%7C_Data_Science_2022_Semana_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Configurações iniciais
Alteração em configurações do Colab para melhor visualização das saídas

In [ ]:
pd.set_option('display.max_columns', 22)

# Carregamento da Base de Dados

A base de dados será carregada diretamente do link do GitHub disponibilizado pela empresa. A base é um arquivo JSON, que precisa ser aberto e tabulado.

O dicionário de dados, com as informações sobre cada coluna da base, pode ser visto [aqui](https://github.com/sthemonica/alura-voz/blob/main/dicionario.md)

In [ ]:
import pandas as pd

df = pd.read_json(
    path_or_buf='https://raw.githubusercontent.com/sthemonica/alura-voz/main/Dados/Telco-Customer-Churn.json',
)

df.head()

,customerID,Churn,customer,phone,internet,account
0,0002-ORFBO,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'One year', 'PaperlessBilling': '..."
1,0003-MKNFE,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
2,0004-TLHLJ,Yes,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
3,0011-IGKFF,Yes,"{'gender': 'Male', 'SeniorCitizen': 1, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
4,0013-EXCHZ,Yes,"{'gender': 'Female', 'SeniorCitizen': 1, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."


Os dados já vêm com uma certa divisão, porém, é possível ver que há mais níveis de profundidade em quase todas as colunas. Então, será utilizado o json_normalize nestas colunas para aprofundar na tabulação dos dados

In [ ]:
# coluna customer
customer = pd.json_normalize(data = df.customer)
customer.head()

,gender,SeniorCitizen,Partner,Dependents,tenure
0,Female,0,Yes,Yes,9
1,Male,0,No,No,9
2,Male,0,No,No,4
3,Male,1,Yes,No,13
4,Female,1,Yes,No,3


In [ ]:
# coluna phone
phone = pd.json_normalize(data = df.phone)
phone.head()

,PhoneService,MultipleLines
0,Yes,No
1,Yes,Yes
2,Yes,No
3,Yes,No
4,Yes,No


In [ ]:
# coluna internet
internet = pd.json_normalize(data = df.internet)
internet.head()

,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
0,DSL,No,Yes,No,Yes,Yes,No
1,DSL,No,No,No,No,No,Yes
2,Fiber optic,No,No,Yes,No,No,No
3,Fiber optic,No,Yes,Yes,No,Yes,Yes
4,Fiber optic,No,No,No,Yes,Yes,No


In [ ]:
# coluna account
account = pd.json_normalize(data = df.account)
account.head()

,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total
0,One year,Yes,Mailed check,65.6,593.3
1,Month-to-month,No,Mailed check,59.9,542.4
2,Month-to-month,Yes,Electronic check,73.9,280.85
3,Month-to-month,Yes,Electronic check,98.0,1237.85
4,Month-to-month,Yes,Mailed check,83.9,267.4


Concatenando todas as colunas em um único DataFrame

In [ ]:
# concatenação lateral (axis=1)
data = pd.concat([df[['customerID',	'Churn']], customer, phone, internet, account], axis = 1)

data.head()

,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,No,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,No,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.4
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,No,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,No,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.4


# Entendendo o DataFrame

Vamos ter uma noção inicial da base de dados, quais informações ela possui e os tipos de dados em cada coluna.

In [ ]:
# quantidade de registros e colunas
print(f'A base de dados possui {data.shape[0]} registros e {data.shape[1]} colunas.')

A base de dados possui 7267 registros e 21 colunas.


In [ ]:
# informações sobre as colunas
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7267 non-null   object 
 1   Churn             7267 non-null   object 
 2   gender            7267 non-null   object 
 3   SeniorCitizen     7267 non-null   int64  
 4   Partner           7267 non-null   object 
 5   Dependents        7267 non-null   object 
 6   tenure            7267 non-null   int64  
 7   PhoneService      7267 non-null   object 
 8   MultipleLines     7267 non-null   object 
 9   InternetService   7267 non-null   object 
 10  OnlineSecurity    7267 non-null   object 
 11  OnlineBackup      7267 non-null   object 
 12  DeviceProtection  7267 non-null   object 
 13  TechSupport       7267 non-null   object 
 14  StreamingTV       7267 non-null   object 
 15  StreamingMovies   7267 non-null   object 
 16  Contract          7267 non-null   object 


Temos 7267 registros. Nenhuma coluna possui dados nulos. A maioria das colunas contém dados categóricos, com a exceção sendo em:


*   Charges.Monthly, que é float64;
*   SeniorCitizen e tenure, marcados como int64.

Baseado no dicionário de dados, o tipo está consistente em quase todas as colunas, exceto em `SeniorCitizen` e `Charges.Total`. 

`SeniorCitizen` indica se o cliente tem idade maior ou igual a 65 anos. Ou seja, era esperado que seus valores fossem strings ('Yes' ou 'No') e não números inteiros. Será feita uma [análise dos valores](#scrollTo=nW733M4oVvMF) na coluna para um possível tratamento e conversão para string (dtype object).

`Charges.Total` contém o valor total gasto pelo cliente, o que se supõe que deveria ser um valor numérico, igual à coluna `Charges.Montly` (valor mensal gasto pelo cliente). No entanto, seu dtype está marcado como object. Talvez alguns valores inválidos foram registrados nela e por isso ela foi classificada como object. Será feita uma [conversão da coluna para float64](#scrollTo=vghLW8UHoKH-&line=1&uniqifier=1) e  os erros de conversão serão marcados como NaN para serem posteriormente analisados e tratados.



## Resolvendo a inconsistência em `SeniorCitizen`

In [ ]:
# verificando os valores contidos na coluna
data['SeniorCitizen'].value_counts()

0    6085
1    1182
Name: SeniorCitizen, dtype: int64

A coluna `SeniorCitizen` possui somente dois valores, 0 e 1, que é outra forma de representação para as respostas como No e Yes. 

Como futuramente essa coluna pode ser utilizada como variável para o modelo de análise dos dados, etapa em que os dados categóricos passam por um processo de encoding, **não farei nenhuma conversão nos valores**, mantendo-os como int64.

In [ ]:
# caso fosse decidido fazer a conversão da coluna para um dtype object, bastaria executar o código abaixo
# data['SeniorCitizen'] = data['SeniorCitizen'].apply(lambda x : 'Yes' if x == 1 else 'No')

## Resolvendo a inconsistência em `Charges.Total`

In [ ]:
# verificando algumas estatísticas da coluna Charges.Total
data['Charges.Total'].describe()

count     7267
unique    6531
top           
freq        11
Name: Charges.Total, dtype: object

É possível observar que 11 "valores" se repetem, sendo o restante valores distintos.

In [ ]:
# convertendo o conteúdo da coluna para float. O coerce marca os erros de conversão como sendo NaN
charges_total = pd.to_numeric(data['Charges.Total'], errors = 'coerce')

charges_total.dtype

dtype('float64')

In [ ]:
# verificando algumas estatísticas dos dados convertidos
charges_total.describe()

count    7256.000000
mean     2280.634213
std      2268.632997
min        18.800000
25%       400.225000
50%      1391.000000
75%      3785.300000
max      8684.800000
Name: Charges.Total, dtype: float64

In [ ]:
# verificando quantos registros não eram numéricos e foram marcados como NaN
charges_total.isnull().sum()

11

Na variável que foi criada, `charges_total`, vemos que 11 registros possuíam conteúdo não-numérico. Curiosamente, é o mesmo número de registros que tinham valores repetidos quando foi aplicado o método describe na coluna. Como são poucos registros, vamos olhar o conteúdo original deles na base.

In [ ]:
# indices que contém NaN
nan_index = charges_total[charges_total.isnull()].index

# selecionando os registros na base para ver o conteúdo original
data.iloc[nan_index]['Charges.Total'].values

array([' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
      dtype=object)

Como podemos observar, nos 11 registros há uma string com espaço em branco `' '`.

`Charges.Total` contabiliza o total gasto pelo cliente desde que assinou seu plano. Uma alternativa para sanar a inconsistência nestes 11 registros é fazer a multiplicação entre as colunas `tenure` e `Charges.Monthly`, obtendo uma aproximação do valor total gasto (talvez não seja totalmente acurado, pois não é possível saber se o valor mensal é fixo ou se pode variar). 



In [ ]:
# Verificando as colunas tenure e Charges.Montly dos 11 registros
data.iloc[nan_index][['tenure', 'Charges.Monthly']]

,tenure,Charges.Monthly
975,0,56.05
1775,0,20.00
1955,0,61.90
2075,0,19.70
2232,0,20.25
2308,0,25.35
2930,0,73.35
3134,0,25.75
3203,0,52.55
4169,0,80.85


É possível notar que o valor de tenure nestes 11 registros é zero. Ou seja, são novos clientes, que ainda não completaram 1 mês de serviço. Podemos supor que foi esse o motivo de a coluna `Charges.Total` estar com uma string com espaço em branco. 

Assumindo que são de fato novos clientes, irei então atribuir à coluna `Charges.Total` o valor que está na coluna `Charges.Monthly`.

In [ ]:
# preenchendo os valores vazios com os valores mensais na variável charges_total
charges_total.fillna(data['Charges.Monthly'], inplace=True)

# verificando se ficou algum registro nulo
charges_total.isnull().sum()

0

In [ ]:
# substituindo os valores na base pelos valores da variável charges_total
data['Charges.Total'] = charges_total

# conferindo a alteração
data.iloc[nan_index][['Charges.Monthly', 'Charges.Total']]

,Charges.Monthly,Charges.Total
975,56.05,56.05
1775,20.00,20.00
1955,61.90,61.90
2075,19.70,19.70
2232,20.25,20.25
2308,25.35,25.35
2930,73.35,73.35
3134,25.75,25.75
3203,52.55,52.55
4169,80.85,80.85


In [ ]:
# conferindo o dtype da coluna 
data['Charges.Total'].dtype

dtype('float64')

## Analisando e resolvendo inconsistências nas variáveis categóricas

Vamos agora verificar os valores contidos em cada variável categórica e se estão consistentes.

In [ ]:
# selecionando as colunas categóricas
categorical_cols = data.select_dtypes(include=object).columns

# verificando os valores únicos de cada coluna categórica
for col in categorical_cols:
  print(f'Coluna: {col}\nValores únicos: {data[col].unique()}\n')

Coluna: customerID
Valores únicos: ['0002-ORFBO' '0003-MKNFE' '0004-TLHLJ' ... '9992-UJOEL' '9993-LHIEB'
 '9995-HOTOH']

Coluna: Churn
Valores únicos: ['No' 'Yes' '']

Coluna: gender
Valores únicos: ['Female' 'Male']

Coluna: Partner
Valores únicos: ['Yes' 'No']

Coluna: Dependents
Valores únicos: ['Yes' 'No']

Coluna: PhoneService
Valores únicos: ['Yes' 'No']

Coluna: MultipleLines
Valores únicos: ['No' 'Yes' 'No phone service']

Coluna: InternetService
Valores únicos: ['DSL' 'Fiber optic' 'No']

Coluna: OnlineSecurity
Valores únicos: ['No' 'Yes' 'No internet service']

Coluna: OnlineBackup
Valores únicos: ['Yes' 'No' 'No internet service']

Coluna: DeviceProtection
Valores únicos: ['No' 'Yes' 'No internet service']

Coluna: TechSupport
Valores únicos: ['Yes' 'No' 'No internet service']

Coluna: StreamingTV
Valores únicos: ['Yes' 'No' 'No internet service']

Coluna: StreamingMovies
Valores únicos: ['No' 'Yes' 'No internet service']

Coluna: Contract
Valores únicos: ['One year' 'Month-

É possível observar que na coluna `Churn` há valores vazios `''`. Será necessário [analisar melhor](#scrollTo=vj_q9Gmp076m) os registros com esse valor vazio.

Também podemos observar que várias colunas possuem o valor "No internet service" e uma delas possui o valor "No phone service". Será que poderíamos substituir esses valores pela resposta "No", ficando assim somente com "Yes" e "No"? Será também que as respostas "No phone/internet service" estão consistentes com o valor contido nas colunas `PhoneService` e `InternetService` para cada registro? Será [necessário verificar](#scrollTo=m8I39hyFk3D2).

### Resolvendo a inconsistência em `Churn`

In [ ]:
# verificando a proporção de valores vazios
print(f'Total de registros: {data.shape[0]}')
print(f'Registros sem valor na coluna Churn: {data[data.Churn == ""].shape[0]}')
print(f'Proporção: {round(data[data.Churn == ""].shape[0] / data.shape[0] * 100, 2)}%')

Total de registros: 7267
Registros sem valor na coluna Churn: 224
Proporção: 3.08%


Temos que 224 registros não possuem nada na coluna `Churn`, o que dá 3% do total de registros na base. Será que removê-los afetaria as análises de maneira significante? *Ainda não tenho maturidade em Data Science o suficiente para dizer*...

Vou **optar por eliminar estes dados**. No entanto, irei **manter uma versão da base com esses dados**, caso futuramente eu decida por tratá-los de outra forma para serem aplicados ao modelo a ser construído.

In [ ]:
# copiando a base como "backup"
data_empty_churn = data.copy()

data_empty_churn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7267 non-null   object 
 1   Churn             7267 non-null   object 
 2   gender            7267 non-null   object 
 3   SeniorCitizen     7267 non-null   int64  
 4   Partner           7267 non-null   object 
 5   Dependents        7267 non-null   object 
 6   tenure            7267 non-null   int64  
 7   PhoneService      7267 non-null   object 
 8   MultipleLines     7267 non-null   object 
 9   InternetService   7267 non-null   object 
 10  OnlineSecurity    7267 non-null   object 
 11  OnlineBackup      7267 non-null   object 
 12  DeviceProtection  7267 non-null   object 
 13  TechSupport       7267 non-null   object 
 14  StreamingTV       7267 non-null   object 
 15  StreamingMovies   7267 non-null   object 
 16  Contract          7267 non-null   object 


In [ ]:
# eliminando os registros com Churn vazio 
print('Registros antes da eliminação: ', data.shape[0])
data = data[data['Churn'] != '']
print('Registros após eliminação: ', data.shape[0])

Registros antes da eliminação:  7267
Registros após eliminação:  7043


In [ ]:
# confirmando os valores na coluna Churn
data['Churn'].unique()

array(['No', 'Yes'], dtype=object)

## Analisando as respostas "No phone service" e "No internet service"

A resposta "No phone service" deveria aparecer somente para os registros que possuem "No" na coluna `PhoneService`. Vamos verificar se há dados que não estão em conformidade com essa lógica, isto é, se há algum registro que possua uma linha telefonica e ao mesmo tempo possua a resposta "No phone service". A mesma lógica será seguida para a coluna `InternetService`.

In [ ]:
# se PhoneService é Yes, MultipleLines não pode ser "No phone service"
data.query("PhoneService == 'Yes' and MultipleLines == 'No phone service'")

,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total


In [ ]:
# se InternetService é Yes, as colunas que constam na query não podem ser "No internet service"
# sim, a query ficou feia... :(
data.query("InternetService == 'Yes' and (OnlineSecurity == 'No internet service' or OnlineBackup == 'No internet service' or DeviceProtection == 'No internet service' or TechSupport == 'No internet service' or StreamingTV == 'No internet service' or StreamingMovies == 'No internet service')")

,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total


As duas consultas não retornaram nenhum registro, então, **podemos assumir que os dados estão em conformidade**.

Não consigo ver a necessidade de ter uma resposta "No phone service" ou "No internet service". Isso cria uma terceira categoria de resposta que, acredito eu, teria o mesmo valor de uma resposta "No". Isso, no entanto, não se configura uma inconsistênca, por isso, **decidi não fazer nenhuma alteração por enquanto**.

# Adaptação da base para PT-BR

As colunas e os valores categóricos da base de dados estão em inglês. Como toda essa análise e futuro relatório estão em português, a empresa solicitou tradução da informação, de modo a ter uma entrega final mais legível para todos os interessados. Desse modo, irei adaptar para o portugues as colunas e valores necessários para que tenham um conteúdo mais explicativo. Para evitar problemas com caracteres especiais, palavras que recebem pontuação (~, ç, ´, etc) serão utilizadas sem esses caracteres.

## Tradução das colunas

In [ ]:
# obtendo o nome das colunas
data.columns.values

array(['customerID', 'Churn', 'gender', 'SeniorCitizen', 'Partner',
       'Dependents', 'tenure', 'PhoneService', 'MultipleLines',
       'InternetService', 'OnlineSecurity', 'OnlineBackup',
       'DeviceProtection', 'TechSupport', 'StreamingTV',
       'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod',
       'Charges.Monthly', 'Charges.Total'], dtype=object)

In [ ]:
# criando um dicionário para traduzir as colunas
translate = {
    'customerID' : 'cliente_ID', 
    'Churn' : 'cancelou_plano', 
    'gender' : 'genero', 
    'SeniorCitizen' : 'maior_65_anos', 
    'Partner' : 'possui_companheiro_companheira',
    'Dependents' : 'possui_dependentes', 
    'tenure' : 'meses_contrato', 
    'PhoneService' : 'assina_telefone', 
    'MultipleLines' : 'multiplas_linhas_telefonicas',
    'InternetService' : 'tipo_assinatura_internet', 
    'OnlineSecurity' : 'adicional_seguranca_online', 
    'OnlineBackup' : 'adicional_backup_online',
    'DeviceProtection' : 'adicional_protecao_dispositivo', 
    'TechSupport' : 'adicional_suporte_VIP', 
    'StreamingTV' : 'assina_TV_cabo',
    'StreamingMovies' : 'assina_streaming_filmes', 
    'Contract' : 'plano_contratado', 
    'PaperlessBilling' : 'fatura_online', 
    'PaymentMethod' : 'forma_pagamento',
    'Charges.Monthly' : 'gasto_mensal', 
    'Charges.Total' : 'gasto_total'
}

In [ ]:
# aplicando a alteração dos nomes
data.rename(columns = translate, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
data.head()

,cliente_ID,cancelou_plano,genero,maior_65_anos,possui_companheiro_companheira,possui_dependentes,meses_contrato,assina_telefone,multiplas_linhas_telefonicas,tipo_assinatura_internet,adicional_seguranca_online,adicional_backup_online,adicional_protecao_dispositivo,adicional_suporte_VIP,assina_TV_cabo,assina_streaming_filmes,plano_contratado,fatura_online,forma_pagamento,gasto_mensal,gasto_total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,No,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.30
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,No,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.40
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,No,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,No,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.40


## Tradução dos valores categóricos

[Mais acima neste notebook](#scrollTo=i1Vc9fbQui18&line=1&uniqifier=1), foram listados os valores para as colunas categóricas. Aproveitei a listagem para criar o dicionário com as traduções.


In [ ]:
# dicionário com as traduções
translate = {
    'Female': 'Mulher', 
    'Male': 'Homem',
    'Yes': 'Sim', 
    'No': 'Nao', 
    'No phone service' : 'Nao assina telefone', 
    'No internet service' : 'Nao assina internet',
    'One year': 'Um ano', 
    'Month-to-month': 'Mensal', 
    'Two year' : 'Dois anos',
    'Mailed check': 'Cheque por Correio', 
    'Electronic check': 'Cheque eletronico', 
    'Credit card (automatic)': 'Cartao de credito',
    'Bank transfer (automatic)': 'Transferencia bancaria'
}

Antes de aplicar a tradução em todo o DataFrame, é necessário fazer um **tratamento especial** para a coluna `tipo_assinatura_internet`. Seus valores são 'DSL', 'Fiber optic' e 'No'. Neste caso, o 'No' se refere a quem não assina a internet. [Anteriormente](#scrollTo=mP3_JqzfoufZ), eu havia decidido manter esta informação de cliente que não assina internet, então, para manter a consistência, irei traduzir este 'No" para 'Nao assina internet'.

Desse modo, a tradução para a coluna `tipo_assinatura_internet` será feita em separado.

In [ ]:
internet_service = {'DSL': 'Cabo', 'Fiber optic': 'Fibra', 'No': 'Nao assina internet'}

In [ ]:
# traduzindo os valores da coluna tipo_assinatura_internet
data.tipo_assinatura_internet = data.tipo_assinatura_internet.map(internet_service)

In [ ]:
data.tipo_assinatura_internet.unique()

array(['Cabo', 'Fibra', 'Nao assina internet'], dtype=object)

Feito esse tratamento especial, podemos prosseguir com a tradução do restante da base.

In [ ]:
# testando a tradução em outro dataframe 
# test = data.replace(to_replace=translate)
# for col in test.select_dtypes(include=object).columns:
#   print(f'Coluna: {col}\nValores únicos: {test[col].unique()}\n')

In [ ]:
data.replace(to_replace=translate, inplace=True)

In [ ]:
# verificando a tradução para alguns registros aleatórios
data.sample(7)

,cliente_ID,cancelou_plano,genero,maior_65_anos,possui_companheiro_companheira,possui_dependentes,meses_contrato,assina_telefone,multiplas_linhas_telefonicas,tipo_assinatura_internet,adicional_seguranca_online,adicional_backup_online,adicional_protecao_dispositivo,adicional_suporte_VIP,assina_TV_cabo,assina_streaming_filmes,plano_contratado,fatura_online,forma_pagamento,gasto_mensal,gasto_total
2659,3704-IEAXF,Nao,Mulher,0,Sim,Sim,72,Nao,Nao assina telefone,Cabo,Nao,Sim,Nao,Sim,Sim,Sim,Dois anos,Nao,Cartao de credito,53.65,3784.00
951,1338-CECEE,Nao,Homem,0,Sim,Sim,41,Sim,Sim,Cabo,Nao,Sim,Sim,Nao,Sim,Nao,Um ano,Nao,Transferencia bancaria,68.50,2839.95
741,1061-PNTHC,Nao,Mulher,0,Sim,Sim,56,Sim,Sim,Fibra,Sim,Nao,Sim,Sim,Sim,Sim,Mensal,Sim,Cheque por Correio,109.60,5953.00
135,0224-HJAPT,Sim,Homem,0,Nao,Nao,5,Sim,Sim,Cabo,Nao,Nao,Nao,Nao,Nao,Nao,Mensal,Nao,Cheque eletronico,47.15,223.15
2668,3714-XPXBW,Nao,Mulher,0,Nao,Nao,37,Sim,Sim,Cabo,Sim,Sim,Sim,Sim,Sim,Nao,Mensal,Sim,Cartao de credito,78.90,2976.95
1913,2703-AMTUL,Nao,Homem,0,Sim,Sim,36,Sim,Nao,Cabo,Sim,Sim,Nao,Nao,Nao,Nao,Um ano,Nao,Cheque por Correio,53.65,1939.35
1897,2683-BPJSO,Sim,Homem,0,Sim,Nao,29,Sim,Sim,Fibra,Sim,Sim,Nao,Nao,Nao,Nao,Mensal,Sim,Cheque eletronico,84.45,2467.10


# Nova coluna: `gasto_diario`

A empresa solicitou a criação de uma coluna que contabilize os gastos diários e que seja colocada na posição 18 da base. Essa é a posição para a coluna `forma_pagamento`. Para deixar as colunas mais organizadas, **optei por colocar a nova coluna na posição 19**, assim, as três opções de gasto ficam lado a lado na tabela (diário, mensal, anual).

Para o cálculo do valor diário, irei assumir um mês com 30 dias, dividindo o gasto mensal por 30.

In [ ]:
# cálculo dos gastos por dia
charges_daily = round(data['gasto_mensal'] / 30, 2)

# inserção da nova coluna na posição 19
data.insert(19, 'gasto_diario', charges_daily)

In [ ]:
# verificando a base
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7043 entries, 0 to 7266
Data columns (total 22 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   cliente_ID                      7043 non-null   object 
 1   cancelou_plano                  7043 non-null   object 
 2   genero                          7043 non-null   object 
 3   maior_65_anos                   7043 non-null   int64  
 4   possui_companheiro_companheira  7043 non-null   object 
 5   possui_dependentes              7043 non-null   object 
 6   meses_contrato                  7043 non-null   int64  
 7   assina_telefone                 7043 non-null   object 
 8   multiplas_linhas_telefonicas    7043 non-null   object 
 9   tipo_assinatura_internet        7043 non-null   object 
 10  adicional_seguranca_online      7043 non-null   object 
 11  adicional_backup_online         7043 non-null   object 
 12  adicional_protecao_dispositivo  70

# Exportando os dados

Na [análise da coluna `Churn`](#scrollTo=vj_q9Gmp076m), eu decidi fazer uma cópia dos dados para uma base de "backup", pois não tenho certeza se a remoção dos dados vazios nesta coluna podem afetar a modelagem e resultados no futuro.

Por esta razão, irei **exportar os dados em dois arquivos CSV**: um com a base que foi tratada e traduzida, com os dados vazios de churn removidos, e outro que contém o backup com os todos os dados de churn (lembrando que para esta última base não foi feita a tradução nem criada a coluna de gastos diários).

In [ ]:
# index=False para não copiar para o arquivo o index numérico que o Pandas criou para o dataframe
data.to_csv('analise_semana_1.csv', index=False)
data_empty_churn.to_csv('dados_sem_churn_removido.csv', index=False)